## Select crypto symbol and date range

In [ ]:
crypto_ticker = 'X:BTCUSD'
start_date = "2020-12-11"
end_date = "2021-10-5"

#### Imports

In [ ]:
from liualgotrader.common.data_loader import DataLoader
from liualgotrader.common.types import DataConnectorType, TimeScale
import matplotlib.pyplot as plt
from talib import BBANDS, MA_Type
from liualgotrader.fincalcs.resample import resample, ResampleRangeType
%matplotlib inline

#### Load symbol data

In [ ]:
dl = DataLoader(scale = TimeScale.day, connector = DataConnectorType.polygon)

In [ ]:
df = dl[crypto_ticker][start_date:end_date]
df

In [ ]:
bband = BBANDS(
    df.close,
    timeperiod=7,
    nbdevdn=1,
    nbdevup=1,
    matype=MA_Type.SMA,
)

In [ ]:
plt.plot(bband[0], c="red")
plt.plot(bband[1], c="blue")
plt.plot(bband[2], c="green")
plt.xticks(rotation=45)
plt.show()

In [ ]:
ohlc_data = df

trace1 = {
    "x": ohlc_data.index,
    "open": ohlc_data.open,
    "high": ohlc_data.high,
    "low": ohlc_data.low,
    "close": ohlc_data.close,
    "type": "candlestick",
    "name": f"{symbol} bars",
    "yaxis": "y2",
    "showlegend": True,
}
trace2 = [
    {
        "x": bband[indx].index,
        "y": bband[indx],
        "type": "scatter",
        "mode": "lines",
        "line": {"width": 2, "color": "red" if indx==0 else "orange" if indx==1 else "green"},
        "yaxis": "y2",
        "name": "upper" if indx ==0 else "middle" if indx ==1 else "low",

        "showlegend": True,
    }
    for indx in range(0, 3)
]
fig = dict(data=[trace1], layout=dict())

fig["layout"]["plot_bgcolor"] = "rgb(200, 200, 200)"
fig["layout"]["xaxis"] = dict(rangeselector=dict(visible=True))
fig["layout"]["yaxis"] = dict(domain=[0, 0.2], showticklabels=False)
fig["layout"]["yaxis2"] = dict(domain=[0.2, 0.8])
fig["layout"]["legend"] = dict(
    orientation="h",
    y=0.95,
    x=0.3,
    yanchor="bottom",
)
fig["layout"]["margin"] = dict(t=40, b=40, r=40, l=40)
fig["data"] += trace2

rangeselector = dict(
    # visibe = True,
    x=0,
    y=0.9,
    bgcolor="rgba(150, 200, 250, 0.4)",
    font=dict(size=13),
    buttons=list(
        [
            dict(count=1, label="1 yr", step="year"),
            dict(count=3, label="3 mo", step="month", stepmode="backward"),
            dict(count=1, label="1 mo", step="month", stepmode="backward"),
            dict(count=7, label="1 wk", step="day", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig["layout"]["xaxis"]["rangeselector"] = rangeselector

colors = []

for i in range(len(ohlc_data.close)):
    if i != 0:
        if ohlc_data.close[i] > ohlc_data.close[i - 1]:
            colors.append("green")
        else:
            colors.append("red")
    else:
        colors.append("red")

fig["data"].append(
    dict(
        x=ohlc_data.index,
        y=ohlc_data.volume,
        marker=dict(color=colors),
        type="bar",
        yaxis="y",
        name="Volume",
        showlegend=False,
    )
)

f = go.Figure(data=fig["data"], layout=fig["layout"])
f.show()